In [ ]:
!pip install numpy pyradiomics slicerio

In [ ]:
from __future__ import print_function
import six
import os

import radiomics
from radiomics import featureextractor

import slicerio
import json


In [ ]:
imagePath = "../201_sag_3d_mprage.nrrd"
maskPath = "../Segmentation.nrrd"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = "Params.yaml"
print('Parameter file, absolute path:', os.path.abspath(paramPath))

In [ ]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

extractor.enabledFeatures
extractor.enableAllFeatures()
# result = extractor.execute(imagePath, maskPath)

# print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
# print('')
# print('Calculated features')
# for key, value in six.iteritems(result):
#     print('\t', key, ':', value)


In [ ]:
extractor.enableAllImageTypes()
extractor.enabledImagetypes

In [ ]:
segmentation_info = slicerio.read_segmentation_info(maskPath)
number_of_segments = len(segmentation_info['segments'])
number_of_segments
segmentation_info['segments']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
# Read features.csv

df = pd.read_csv("features.csv")
features_list = ['shape', 'firstorder', 'glcm', 'glrlm', 'glszm', 'gldm', 'ngtdm']
image_types = ['original', 'exponential', 'gradient', 'lbp-2D', 'lbp-3D', 'logarithm', 'square', 'squareroot', 'wavelet']
labels_list = [1,2,3]

def plot_graph(df_label_features, dir_name, label):
    # draw a boxplot for each feature in features_for_cistern on one chart
    for column in df_label_features.columns:
        if column != 'image':
            print(column)
            fig = plt.figure(figsize=(5,5))
            # df_label_features.boxplot(column=column)
            # y = df_label_features[column]
            # x = np.random.randint(0,5, size=len(y))
            # plt.plot(x, y, 'r.', alpha=0.3)
            # plt.title('BoxPlot of {} for {}'.format(column,label))

            sns.set(style="whitegrid")
            ax = sns.boxplot(y=column, data=df_label_features, showfliers = False, palette="Set3")
            ax = sns.stripplot(y=column, data=df_label_features, color=".25")
            
            # remove y-axis title
            ax.set(ylabel='')
            plt.title('BoxPlot of {} for {}'.format(column,label))
            plt.savefig(dir_name + '/boxplot_{}.png'.format(column),  bbox_inches = 'tight')
            plt.close(fig)

# Extract features

for label in labels_list:
    print(label)
    df_label = df[(df['label'] == label)]
    for type in image_types:
        print(type)
        for feature in features_list:
            print(feature)
            df_label_features = df_label.filter(regex=type + "_" +feature)
            dir_name = 'label_' + str(label) + '/' + type + '/' + feature
            os.makedirs(dir_name, exist_ok=True)
            plot_graph(df_label_features, dir_name, label)

In [ ]:
# get value of image column for index in features_for_cistern
df = pd.read_csv("features.csv")
feature="firstorder"
df_label = df[(df['label'] == 3)]

df_label_features = df_label.filter(regex= "original_" +feature)
df_label_features['image'] = df.iloc[df_label_features.index]['image']
df_label_features.to_csv("features_for_cc.csv")